# COVID-19 reports parser

The data is fetched from [GitHub](https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data).


In [8]:
import sys
import os
#os.getcwd()
!{sys.executable} -m pip install -r ../parser/requirements.txt

  Using cached https://files.pythonhosted.org/packages/cb/2a/07f87440444fdf2c5870a710b6770d766a1c7df9c827b0c90e807f1fb4c5/python_dotenv-0.13.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/0e/4c/6ece00be1fe8b4b4aada2d6f2ed7b62814b9abfa21962a95464efcd2cb07/PyGithub-1.50-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/2b/ae/1f02f68eaa4aa878f184b2adc20a1923becb80a4da6c76efa33450011902/Deprecated-1.2.9-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/87/8b/6a9f14b5f781697e51259d81657e6048fd31a113229cf346880bb7545565/PyJWT-1.7.1-py2.py3-none-any.whl


In [9]:
import importlib.util

def load_module_from_path(module_name, path):
    """
    Loads a Python module from a path.
    FIXME: parser module should be installed as a package
    """

    spec = importlib.util.spec_from_file_location(module_name, path)
    module = importlib.util.module_from_spec(spec)
    spec.loader.exec_module(module)
    
    return module

In [ ]:
import logging
from github import Github
from pathlib import Path
import time
from datetime import timedelta

parser_module = load_module_from_path('parser', Path('../parser/src/parser.py'))
storage_module = load_module_from_path('storage', Path('../parser/src/storage.py'))

logging.basicConfig(level=logging.INFO)

# NOTE: Please update GitHub access token
github = Github('db38e8cbf731f8d03cff6b9a15300f68486dcd69')
# NOTE: Please update database credentials
connection_config = storage_module.StorageConnectionConfig(
    driver='postgres',
    host='127.0.0.1',
    port=5432,
    schema='covid_19',
    user='postgres',
    password='root'
)
storage = storage_module.Storage(connection_config)

while True:
    last_update_date = storage.get_last_update_date()
    
    if last_update_date:
        # We need to remove last available report to eliminate possibility of inserting duplicates for this day
        # FIXME: It's bad in terms of perfomance but it's the simplest solution
        storage.delete_report(last_update_date)

    parser = parser_module.Parser(github, last_update_date)

    for status_report in parser.parse():
        storage.save_daily_report(status_report)

    sleep_timeout = timedelta(hours=1)
    
    logging.info(f'Sleeping for {sleep_timeout}')

    time.sleep(sleep_timeout.total_seconds())

INFO:storage:Started fetching information about the last update date
INFO:storage:Started creating a new connection
INFO:storage:New connection has been successfully created
INFO:storage:Started creating a database schema (if it doesn't yet exist)
INFO:storage:Database schema has been successfully created (if it didn't exist before)
INFO:storage:Finished fetching information about the last update date. Database was updated last time on None
INFO:storage:Started saving a daily report to the database
INFO:storage:Successfully finished saving a daily report to the database
INFO:storage:Started saving a daily report to the database
INFO:storage:Successfully finished saving a daily report to the database
INFO:storage:Started saving a daily report to the database
INFO:storage:Successfully finished saving a daily report to the database
INFO:storage:Started saving a daily report to the database
INFO:storage:Successfully finished saving a daily report to the database
INFO:storage:Started saving

INFO:storage:Started saving a daily report to the database
INFO:storage:Successfully finished saving a daily report to the database
INFO:storage:Started saving a daily report to the database
INFO:storage:Successfully finished saving a daily report to the database
INFO:storage:Started saving a daily report to the database
INFO:storage:Successfully finished saving a daily report to the database
INFO:storage:Started saving a daily report to the database
INFO:storage:Successfully finished saving a daily report to the database
INFO:storage:Started saving a daily report to the database
INFO:storage:Successfully finished saving a daily report to the database
INFO:storage:Started saving a daily report to the database
INFO:storage:Successfully finished saving a daily report to the database
INFO:storage:Started saving a daily report to the database
INFO:storage:Successfully finished saving a daily report to the database
INFO:storage:Started saving a daily report to the database
INFO:storage:Succ